This is a link budget for a simple one directional free space optical link

# Imports

In [25]:
import numpy as np
import scipy.constants as constants
import time
from chart_studio import plotly as plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as graphs


# Optics library
Simple sequential raytracing, vectorized as much as possible for fast execution

In [62]:
class RayBundle:
    def __init__(self, origins, directions, powers, index):
        """ Create a RayBundle object
        Args:
          origins: a N x 3 numpy array representing the origins of the rays
          directions: a N x 3 numpy array representing the directions of the rays (unit vectors)
          powers: a N x 1 numpy array representing the power of the rays"""
        self.origins = origins
        self.directions = directions
        self.powers = powers

class Aperture:
    """ Apertures are limits in local (x, y) coordinates that can be used to filter rays
    A derived class should implement the contains function, which checks if a point is inside the aperture."""
    def contains(self, points):
        """ Check if a point is inside the aperture
        Args:
          points: a N x 2 numpy array representing the point in 2D (surface coordinates) space
        Returns:
          a N x 1 numpy array of booleans representing if the points are inside the aperture"""
        raise NotImplementedError("contains() is not implemented")
    def mesh(self):
        """ Generate a mesh of points that are inside the aperture. Used to display the surfaces
        Returns:
          a N x 2 numpy array representing the points inside the aperture, generated from any parameter space.
          best is to use the numpy mgrid function for this."""
        raise NotImplementedError("mesh() is not implemented")

class Surface:
    """ Surfaces are objects that can propagate RayBundles. They are the basic building blocks of optical systems.
    A derived class should implement the sag function, which calculates the sag of the surface at a given point."""
    def __init__(self, origin, rotation, aperture, index1, index2):
        """ Create a Surface object
        Args:
          origin: 1 x 3 numpy array representing the origin of the surface in 3D space
          rotation: 3 x 3 numpy array representing the rotation matrix of the surface in 3D space
          aperture: an Aperture object representing the aperture of the surface
          index1: the index of refraction of the medium the light is coming from
          index2: the index of refraction of the medium the light is going to. If 0, this is a mirror."""
        self.origin = origin
        self.rotation = rotation
        self.aperture = aperture
        self.index1 = index1
        self.index2 = index2

    def normals(self, points):
        """ Calculate the normal of the surface at a given point
        Args:
          points: a N x 2 numpy array representing the point in 2D (surface coordinates) space
        Returns:
          a N x 2 numpy array representing the normal (in surface coordinates) of the surface at these points"""
        # We are going to use the gradient of the sag function to calculate the normal.
        # For this we need to choose a good epsilon in x and y, not too small to avoid numerical errors, but not too large
        # to avoid sampling errors for a high frequency sag function. Choosing something close to the wavelength of light
        # is a good compromise.
        epsilon = 100e-9
        sag = self.sag(points)
        dsag_dx = (self.sag(points + np.array([epsilon, 0])) - sag) / epsilon
        dsag_dy = (self.sag(points + np.array([0, epsilon])) - sag) / epsilon
        v1 = np.array([dsag_dx, 0, 1])
        v2 = np.array([0, dsag_dy, 1])
        normals = np.zeros((points.shape[0], 2))
        normals[:, 0] = epsilon
        normals[:, 1] = epsilon
        normals[:, 0] = -sag_x
        # Given that dimensions are in meters, we can use a delta
        # in t


    def intersections(self, ray_bundle):
        """ Calculate the intersection of a RayBundle with the surface
        Args:
          ray_bundle: a RayBundle object
        Returns:
          a 2D numpy array representing the intersection of the rays with the surface in 2D (surface coordinates) space"""
        # First we transform the ray bundle 
        # We are going to use a Newton-Raphson method to find the intersection of the rays with the surface
        # We start by assuming that the intersection is at the origin of the surface
        origins = ray_bundle.origins
        directions = ray_bundle.directions
        origin = self.origin
        normal = self.normal


        # Calculate the intersection point of the rays with the surface
        t = np.sum((origin - origins) * normal, axis=1) / np.sum(directions * normal, axis=1)
        intersection_points = origins + t[:, np.newaxis] * directions
        return intersection_points


    def refract(self, ray_bundle):
        """ Propagate a RayBundle going through the surface
        Args:
          ray_bundle: a RayBundle object
        Returns:
          a RayBundle object"""
        
        raise NotImplementedError("propagate() is not implemented")


class CircularAperture(Aperture):
    """ A CircularAperture is an aperture that is a circle"""
    def __init__(self, radius):
        """ Create a CircularAperture object
        Args:
          radius: the radius of the aperture"""
        self.radius = radius

    def contains(self, points):
        """ Check if a point is inside the aperture
        Args:
          points: a N x 2 numpy array representing the point in 2D space
        Returns:
          a N x 1 numpy array of booleans representing if the points are inside the aperture"""
        return np.norm(points, axis=1) < self.radius

    def mesh(self):
        """ Generate a mesh of points that are inside the aperture. Used to display the surfaces.
        Returns:
          mesh_x: The x coordinates of the mesh points, as a 2D array
          mesh_y: The y coordinates of the mesh points, as a 2D array"""
        thetas = np.linspace(0, 2 * np.pi, 30)
        rs = np.linspace(0, self.radius, 15)
        thetagrid, rgrid = np.meshgrid(thetas, rs)
        mesh_x = rgrid * np.cos(thetagrid)
        mesh_y = rgrid * np.sin(thetagrid)
        return mesh_x, mesh_y

class SphericalSurface(Surface):
    """ A SphericalSurface is a Surface that has a spherical sag function"""
    def __init__(self, origin, rotation, aperture, index1, index2, radius):
        """ Create a SphericalSurface object
        Args:
          origin: 1 x 3 numpy array representing the origin of the surface in 3D space
          rotation: 3 x 3 numpy array representing the rotation matrix of the surface in 3D space
          aperture: an Aperture object representing the aperture of the surface
          index1: the index of refraction of the medium the light is coming from
          index2: the index of refraction of the medium the light is going to. If 0, this is a mirror.
          radius: the radius of the sphere"""
        super().__init__(origin, rotation, aperture, index1, index2)
        self.radius = radius

    def sag(self, points):
        """ Calculate the sag of the surface for an aray of 2D points
        Args:
          points: a N x 2 numpy array representing the point in 2D (surface coordinates) space
        Returns:
          a N x 1 numpy array representing the sag of the surface at these points"""
        r2 = points[:, 0]**2 + points[:, 1]**2
        return self.radius - np.sqrt(self.radius**2 - r2)

    def mesh(self):
        """ Generate a mesh representing the surface. Used to display the surfaces
        Returns:
          mesh_x: The x coordinates of the mesh points, as a 2D array
          mesh_y: The y coordinates of the mesh points, as a 2D array
          mesh_z: The z coordinates of the mesh points, as a 2D array"""
        mesh_x, mesh_y = self.aperture.mesh()
        xs = mesh_x.flatten()
        ys = mesh_y.flatten()
        points2d = np.array([xs, ys]).T
        zs = self.sag(points2d)
        mesh_z = np.reshape(zs, mesh_x.shape)
        return mesh_x, mesh_y, mesh_z

    def refract(self, ray_bundle):
        """ Propagate a RayBundle going through the surface
        Args:
          ray_bundle: a RayBundle object
        Returns:
          a RayBundle object"""
        origins = ray_bundle.origins
        directions = ray_bundle.directions
        powers = ray_bundle.powers
        index1 = self.index1
        index2 = self.index2
        radius = self.radius
        origin = self.origin
        normal = self.normal

        # Calculate the intersection point of the rays with the sphere
        a = np.sum(directions**2, axis=1)
        b = 2 * np.sum((origins - origin) * directions, axis=1)
        c = np.sum((origins - origin)**2, axis=1) - radius**2
        discriminant = b**2 - 4 * a * c
        t = (-b - np.sqrt(discriminant)) / (2 * a)
        intersection_points = origins + t[:, np.newaxis] * directions

        # Calculate the normal at the intersection points
        normals = intersection_points - origin
        normals /= np.linalg.norm(normals)
                                  

# Create a spherical surface with a circular aperture
origin = np.array([0, 0, 0])
rotation = np.eye(3)
aperture = CircularAperture(1)
index1 = 1
index2 = 1.5
radius = 1.5
surface = SphericalSurface(origin, rotation, aperture, index1, index2, radius)
x, y, z = surface.mesh()
graph_data = [graphs.Surface(x=x, y=y, z=z, colorscale='Viridis')]

line_format = dict(color='black', width=2)
range_x = list(range(0, x.shape[0], 3))
if x.shape[0] - 1 not in range_x:
  range_x.append(x.shape[0] - 1)
for i in range_x:
  graph_data.append(graphs.Scatter3d(x=x[i, :], y=y[i, :], z=z[i, :], mode='lines', line=line_format, showlegend=False))
range_y = list(range(0, x.shape[1], 5))
if x.shape[1] - 1 not in range_y:
  range_y.append(x.shape[1] - 1)
for i in range_y:
  graph_data.append(graphs.Scatter3d(x=x[:, i], y=y[:, i], z=z[:, i], mode='lines', line=line_format, showlegend=False))



layout = graphs.Layout(
    title='Surface sag',
    autosize=False,
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            showbackground=False,
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
        ),
        aspectmode='data',
    )
)
fig = graphs.Figure(data=graph_data, layout=layout)




iplot(fig)



# Transmit

Possible choice for source laser:
Rohm single mode 220mW NIR laser. Available in 820nm, 840nm and 850nm.

[Rohm 850nm laser](https://fscdn.rohm.com/en/products/databook/datasheet/opto/laser_diode/infrared/rld85nzj400a008-e.pdf)

In [2]:
tx_power_W = 0.22 #@param {"type":"number"}
tx_wavelength_m = 850e-9 #@param {"type":"number"}
tx_fast_axis_divergence_deg = 9.5 #@param {"type":"number"}
tx_slow_axis_divergence_deg = 19 #@param {"type":"number"}


In [3]:

def laser_source(
    wavelength_m,
    power_w,
    num_rays,
    div_deg,
    label="",
    draw_color=None):
  """ Ray bundle for a single mode laser.

  The emitting area is simulated as a point source. It should be modeled as a
  gaussian beam waist, but because we are going to defocus the beam on purpose,
  it should not make too much of a difference.

  Parameters:
    wavelength_m: The source wavelength, in meters,
    power_w: The source total power, in watts,
    num_rays: Tuple (nx,ny) with the number of rays used to create the beam.
    div_deg: Tuple with the beam's horizontal and vertical divergence.
    label: String used to identify the ray source
    draw_color: Color used to represent the rays in plots. Default (None)
                automatically chooses a color based on the wavelength.
                Otherwise, can be any valid matplotlib color descriptor. See :
                https://matplotlib.org/3.3.1/api/colors_api.html

  Returns:
    A list of ray objects.
  """
  nx, ny = num_rays
  dx, dy = div_deg

  # We are going to sample the beam uniformly, and assign a varying power to
  # each ray. This will allow us to better simulate what happens at the edge
  # of the lens



#   ret_val = []

#    nx, ny = num_rays
#    dx, dy = size

#    # note modify this to use traits
#    dx = float(dx)
#    dy = float(dy)

#    for ix in range(nx):
#        for iy in range(ny):
#            x = -dx / 2.0 + dx * ix / (nx - 1)
#            y = -dy / 2.0 + dy * iy / (ny - 1)
#            ret_val.append(
#                Ray(
#                    origin=(x, y, 0),
#                    direction=(0, 0, 1),
#                    wavelength=wavelength,
#                    label=label,
#                    draw_color=draw_color,
#                ).ch_coord_sys_inv(origin, direction)
#            )
#    return ret_val


L=library.Edmund["32475"]
SEN=CCD(size=(20,20))#c.rotateZ(D[2])
    #c.rotateY(D[1])
    #c.rotateX(D[0])

R=parallel_beam_c(origin=(0.0, 0.0, 0.0), direction=(0.0, 0.0, 0.0),
                  size=(10, 10), num_rays=(200, 200), wavelength=0.58929)
S=System(complist=[(L,(0,0,50),(0,0,0)),(SEN,(0,0,150),(0,0,0))],n=1)
tic = time.time()
#S._np_rays += R
S.ray_add(R)
toc = time.time()
print(toc - tic)
tic = time.time()
S.propagate()
toc = time.time()
print(toc - tic)
#renderer = Plot3D(S)
#display(renderer)



NameError: name 'library' is not defined

# Optical budget
Both TX and RX are simple collimators with a limited divergent beam. We assume that the beams are both uniform in angular distribution and spatially, sampled at the waist.

In [3]:
# Parameters
link_distance_m = 1000 #@param {"type":"number"}
rx_aperture_diameter_m = 0.05 #@param {"type":"number"}
rx_f_number = 1 #@param {"type":"number"}
rx_optical_transmission = 0.8 #@param {"type":"number"}
rx_apd_diameter_m = 500e-9 # @param {"type":"number"}
tx_divergence_rad = 10e-3 #@param {"type":"number"}
tx_optical_transmission = 0.8 #@param {"type":"number"}
tx_aperture_diameter_m = 0.05 #@param {"type":"number"}

# Beam divergences
rx_focal_length_m = rx_aperture_diameter_m * rx_f_number
rx_divergence_rad = rx_apd_diameter_m / rx_focal_length_m
tx_divergence_deg = np.degrees(tx_divergence_rad)
rx_divergence_deg = np.degrees(rx_divergence_rad)

# Aperture surface area
rx_aperture_area_m2 = np.pi * (rx_aperture_diameter_m / 2)**2
tx_aperture_area_m2 = np.pi * (tx_aperture_diameter_m / 2)**2
# Beam footprint at full the receiver
rx_beam_diameter_m = link_distance_m * tx_divergence_rad
rx_beam_area_m2 = np.pi * (rx_beam_diameter_m / 2)**2
# Full transmission
link_transmission = tx_optical_transmission * rx_optical_transmission * rx_aperture_area_m2 / rx_beam_area_m2
rx_power_W = tx_power_W * link_transmission
# Link loss (dBm)
link_loss_dBm = 10 * np.log10(link_transmission)

# Display results
print("TX")
print(f"Power: {tx_power_W:.2g} W")
print(f"Wavelength: {tx_wavelength_m * 1e9} nm")
print(f"Beam divergence: {tx_divergence_rad * 1e3:.2f}mrad (+/-{tx_divergence_deg/2:.2f} deg)")
print("")

print("RX")
print(f"Beam divergence: {rx_divergence_rad * 1e3:.2f}mrad (+/-{rx_divergence_deg/2:.2f} deg)")
print(f"Beam diameter: {rx_beam_diameter_m:.2f} m")
print(f"Power on detector: {rx_power_W:.2g} W")
print("")

print("Link")
print(f"Link loss: {link_loss_dBm:.2f} dBm")



TX
Power: 0.22 W
Wavelength: 850.0 nm
Beam divergence: 10.00mrad (+/-0.29 deg)

RX
Beam divergence: 0.01mrad (+/-0.00 deg)
Beam diameter: 10.00 m
Power on detector: 3.5e-06 W

Link
Link loss: -47.96 dBm


In [4]:

# Link parameters
link_rate_bps = 3e9 #@param
link_spectral_efficiency = 2 #@param

# Transmit parameters

# Receive parameters
rx_apd_capacitance_F = 1.2e-12 #@param
rx_apd_gain_ApW = 65 #@param
rx_tia_input_impedance_ohm = 50 #@param
rx_tia_transimpedance_ohm = 1000 #@param


# RX Signal
- The receiver is composed of a silicon APD, followed by a TIA
- We assume that the TIA has more bandwidth than the APD. The bandwidth limiting factor is the first order low pass formed by the APD capacitance and TIA input impedance.


In [5]:
# Number of photons per bit
photon_energy_J = constants.h * constants.c / tx_wavelength_m
num_photons_per_bit = rx_power_W / (photon_energy_J * link_rate_bps)
# Signal amplitude (assuming DC balance)
rx_amplitude_W = rx_power_W / 2
rx_amplitude_A = rx_power_W * rx_apd_gain_ApW
rx_amplitude_V = rx_amplitude_A * rx_tia_transimpedance_ohm

# Input bandwidth
rx_bandwidth_Hz = 1 / (rx_tia_input_impedance_ohm * rx_apd_capacitance_F * 2 * np.pi)
rx_required_spectral_efficiency = link_rate_bps / rx_bandwidth_Hz

# Display results
print(f"Power on detector: {rx_power_W:.2g} W")
print(f"Number of photons per bit: {num_photons_per_bit:.2f}")
print(f"Minimal required spectral efficiency (NRZ OOK is 2): {rx_required_spectral_efficiency:.2f}")
print(f"Signal amplitude: {rx_amplitude_V:.2f} V")
print(f"APD Bandwidth: {rx_bandwidth_Hz / 1e9:.2f} GHz")



Power on detector: 3.5e-06 W
Number of photons per bit: 5020.69
Minimal required spectral efficiency (NRZ OOK is 2): 1.13
Signal amplitude: 0.23 V
APD Bandwidth: 2.65 GHz


# Eye safety
- We use IEC 60825.1:2014, which has been adopted by the FDA as regulating eye safety standard in the US. A copy of the doc can be purchased directly from the IEC, or the Australian version (which is identical to the US version) can be downloaded directly from [here](https://research.unsw.edu.au/document/1-2014%20Safety%20of%20laser%20products_Equipment%20classification%20and%20requirements.pdf)
- To simplify the analysis we assume that the beam has the worst possible angular subtense (it will focus perfectly on in one spot on the retina). Additional margin could be extracted by considering the fact that the beam is actually divergent (factor C6 will not be 1).
- We assume a wavelength range of 700nm to 1050nm. AEL formulas will be different for wavelengths outside of this range.
- We assume a uniform beam. If the TX beam is non uniform, power will be higher through the limiting aperture !
- Relevant table sin IEC 60825:
 - Table 3: AEL formulas
 - Table 9: Values for parameters C4 and C7
 - Table A.6: Limiting aperture size
- We assume that the signal is 8b/10b encoded, so average power is 1/2 of peak power and that the maximum run length is 5.


In [6]:
# Compute the AEL.
# C4 (see table 9)
eye_c4 = 10**(0.002 * (tx_wavelength_m * 1e9 - 700))
# C7 (see table 9)
eye_c7 = 1
# AEL for CW conditions (see table 3)
eye_ael_cw_w = 3.9e-4 * eye_c4 * eye_c7
# AEL for each bit (1 pulse)
eye_ael_pulse_j = 7.7e-8 * eye_c4
# Limiting aperture (see table A.6)
eye_aperture_diameter_m = 0.007
eye_aperture_area_m2 = (eye_aperture_diameter_m / 2)**2 * np.pi
# TX peak power through limiting aperture (assuming uniform TX beam)
max_pulse_length = 5
duty_cycle = 0.5
eye_aperture_power_W = duty_cycle * tx_power_W * eye_aperture_area_m2 / tx_aperture_area_m2
eye_aperture_pulse_energy_j = eye_aperture_power_W * max_pulse_length / link_rate_bps

# Display results
print(f"Limiting aperture diameter: {eye_aperture_diameter_m * 1e3:.1f} mm")
print(f"Limiting aperture area: {eye_aperture_area_m2 * 1e6:.2f} mm^2")
print(f"AEL for CW signal: {eye_ael_cw_w * 1e3:.2f} mW")
print(f"TX CW power through limiting aperture ({duty_cycle * 100:.0f}% duty cycle): {eye_aperture_power_W * 1e3:.2f} mW")
print(f"AEL for pulse: {eye_ael_pulse_j * 1e9:.2f} nJ")
print(f"TX max pulse ({max_pulse_length} bits) energy through limiting aperture: {eye_aperture_pulse_energy_j * 1e9:.2g} nJ")


Limiting aperture diameter: 7.0 mm
Limiting aperture area: 38.48 mm^2
AEL for CW signal: 0.78 mW
TX CW power through limiting aperture (50% duty cycle): 2.16 mW
AEL for pulse: 153.64 nJ
TX max pulse (5 bits) energy through limiting aperture: 0.0036 nJ
